In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from konlpy.tag import Kkma, Komoran, Okt, Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from hanspell import spell_checker

In [42]:
# 데이터 불러오기
trade_df = pd.read_excel('유통_무역_운송.xls')
edu_df = pd.read_excel('교육업.xls')
cons_df = pd.read_excel('건설업.xls')

In [43]:
# 필요 없는 열 삭제
trade_df.drop('Unnamed: 0', axis = 1, inplace = True)
edu_df.drop('Unnamed: 0', axis = 1, inplace = True)
cons_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [44]:
# 회사 열을 인덱스로 설정하기
trade_df.set_index('회사', inplace = True)
edu_df.set_index('회사', inplace = True)
cons_df.set_index('회사', inplace = True)

In [46]:
# 중복된 데이터 삭제
trade_df.drop_duplicates(['요약','장점','단점','경영진에게 바라는 점'], inplace = True)
edu_df.drop_duplicates(['요약','장점','단점','경영진에게 바라는 점'], inplace = True)
cons_df.drop_duplicates(['요약','장점','단점','경영진에게 바라는 점'], inplace = True)

In [48]:
# py-hanspell을 이용한 맞춤법 검사 함수 정의
def spelling(df):
    # [요약]부터 [경영진에게 바라는 점]까지
    for col in df.columns[2:]:
        for j in range(df.shape[0]):
            try:
                # 고칠 게 있으면 고치고 기존 데이터 대체
                result_train = spell_checker.check(df[col][j])
                df[col][j] = result_train.as_dict()['checked']
            except:
                # 없으면 pass
                pass

In [49]:
# 데이터 맞춤법 수정
spelling(trade_df)
spelling(edu_df)
spelling(cons_df)

In [50]:
# 지금까지 전처리 한 결과 엑셀 파일로 저장
trade_df.to_excel(excel_writer = '유통_무역_운송_맞춤법.xlsx')
edu_df.to_excel(excel_writer = '교육업_맞춤법.xlsx')
cons_df.to_excel(excel_writer = '건설업_맞춤법.xlsx')

In [209]:
# 데이터 불러오기
trade_df = pd.read_excel('data/spelling/유통_무역_운송_맞춤법.xlsx')
edu_df = pd.read_excel('data/spelling/교육업_맞춤법.xlsx')
cons_df = pd.read_excel('data/spelling/건설업_맞춤법.xlsx')

In [223]:
# null값 확인
print(trade_df.isnull().sum())
print(edu_df.isnull().sum())
print(cons_df.isnull().sum())

날짜             0
직무             0
요약             0
장점             0
단점             0
경영진에게 바라는 점    0
dtype: int64
날짜             0
직무             0
요약             0
장점             0
단점             0
경영진에게 바라는 점    0
dtype: int64
날짜             0
직무             0
요약             0
장점             0
단점             0
경영진에게 바라는 점    0
요약토큰화          0
dtype: int64


In [ ]:
# null값 제거
index = trade_df[trade_df['경영진에게 바라는 점'].isnull()].index
trade_df.drop(index, inplace = True)

In [210]:
# 회사 열을 인덱스로 설정하기
trade_df.set_index('회사', inplace = True)
edu_df.set_index('회사', inplace = True)
cons_df.set_index('회사', inplace = True)

In [211]:
import re

# 특수문자 제거 함수 정의
def clean_text(df):
    for col in df.columns[2:]:
        for j in range(df.shape[0]):
            df[col][j] = re.sub('[-=+,#/\?:^$.@*\"~&%ㆍ!\\‘|\(\)\[\]\<\>`\'…]', ' ', str(df[col][j]))

In [212]:
# 특수문자 제거
clean_text(cons_df)

In [213]:
# 영어 제거 함수 정의
def no_english(df):
    for col in df.columns[2:]:
        df[col] = df[col].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힇]", " ")

In [214]:
# 영어 제거
no_english(cons_df)

In [218]:
# mecab 토큰화 및 불용어 처리
mecab = Mecab()
stopwords = ['은', '는', '이', '가', '도', '다', '의', '한', '에', '게', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '하다', '했다']

cons_df['요약토큰화'] = cons_df['요약'].apply(mecab.morphs)
cons_df['요약토큰화'] = cons_df['요약토큰화'].apply(lambda x: [item for item in x if item not in stopwords])

In [207]:
# Okt 토큰화 및 불용어 처리
okt = Okt()

cons_df['요약토큰화'] = cons_df['요약'].apply(okt.morphs)
cons_df['요약토큰화'] = cons_df['요약토큰화'].apply(lambda x: [item for item in x if item not in stopwords])

In [63]:
# Kkma 토큰화 및 불용어 처리
kkm = Kkma()

cons_df['요약토큰화'] = cons_df['요약'].apply(kkm.morphs)
cons_df['요약토큰화'] = cons_df['요약토큰화'].apply(lambda x: [item for item in x if item not in stopwords])

In [66]:
# Komoran 토큰화 및 불용어 처리
kom = Komoran()

cons_df['요약토큰화'] = cons_df['요약'].apply(kom.morphs)
cons_df['요약토큰화'] = cons_df['요약토큰화'].apply(lambda x: [item for item in x if item not in stopwords])

In [224]:
cons_df.to_excel(excel_writer = '건설업_mecab.xlsx')